In [43]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt
from scipy.stats import skew
from scipy.stats import boxcox
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from scipy.stats import f_oneway

In [44]:
data=pd.read_csv('train.csv')
#data.iloc[:,1][np.isfinite(data.iloc[:,1])]=boxcox(data.iloc[:,1]+52)[0]
#data.iloc[:,1]=(data.iloc[:,1]-data.iloc[:,1].mean())/data.iloc[:,1].std()
data.iloc[:,1].fillna(data.iloc[:,1].mean(), inplace=True)
#data.iloc[:,3][np.isfinite(data.iloc[:,3])]=boxcox(data.iloc[:,3]+0.0325)[0]
#data.iloc[:,3]=(data.iloc[:,3]-data.iloc[:,3].mean())/data.iloc[:,3].std()
data.iloc[:,3].fillna(data.iloc[:,3].mean(), inplace=True)
#data.iloc[:,5][np.isfinite(data.iloc[:,5])]=boxcox(data.iloc[:,5]+650)[0]
#data.iloc[:,5]=(data.iloc[:,5]-data.iloc[:,5].mean())/data.iloc[:,5].std()
data.iloc[:,5].fillna(data.iloc[:,5].mean(), inplace=True)
data.iloc[:,2][data.iloc[:,2]=='LF']='low fat'
data.iloc[:,2][data.iloc[:,2]=='reg']='regular'
data.iloc[:,2]=data.iloc[:,2].str.lower()
data

C:\Users\Parth Patekar\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Parth Patekar\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300000,low fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920000,regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500000,low fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200000,regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930000,low fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395000,regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650000,regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,12.857645,low fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200000,regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200000,regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [45]:
X=data.iloc[:,1:data.shape[1]-1]
y=data.iloc[:,-1]
var_mod=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']
le = LabelEncoder()
for i in var_mod:
    X[i] = X[i].factorize()[0]
    X[i] = le.fit_transform(X[i])
X=X.values
y=y.values
X

array([[  9.30000000e+00,   0.00000000e+00,   1.60473010e-02, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  5.92000000e+00,   1.00000000e+00,   1.92782160e-02, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  1.75000000e+01,   0.00000000e+00,   1.67600750e-02, ...,
          1.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       ..., 
       [  1.06000000e+01,   0.00000000e+00,   3.51862710e-02, ...,
          3.00000000e+00,   2.00000000e+00,   0.00000000e+00],
       [  7.21000000e+00,   1.00000000e+00,   1.45220646e-01, ...,
          1.00000000e+00,   1.00000000e+00,   1.00000000e+00],
       [  1.48000000e+01,   0.00000000e+00,   4.48782800e-02, ...,
          3.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [46]:
lr = linear_model.LinearRegression()
lr.fit(X,y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [47]:
test=pd.read_csv('test.csv')
test=test.iloc[:,:test.shape[1]]
#test.iloc[:,1][np.isfinite(test.iloc[:,1])]=boxcox(test.iloc[:,1]+52)[0]
#test.iloc[:,1]=(test.iloc[:,1]-test.iloc[:,1].mean())/test.iloc[:,1].std()
test.iloc[:,1].fillna(test.iloc[:,1].mean(), inplace=True)
#test.iloc[:,3][np.isfinite(test.iloc[:,3])]=boxcox(test.iloc[:,3]+0.0325)[0]
#test.iloc[:,3]=(test.iloc[:,3]-test.iloc[:,3].mean())/test.iloc[:,3].std()
test.iloc[:,3].fillna(test.iloc[:,3].mean(), inplace=True)
#test.iloc[:,5][np.isfinite(test.iloc[:,5])]=boxcox(test.iloc[:,5]+650)[0]
#test.iloc[:,5]=(test.iloc[:,5]-test.iloc[:,5].mean())/test.iloc[:,5].std()
test.iloc[:,5].fillna(test.iloc[:,5].mean(), inplace=True)
test.iloc[:,2][test.iloc[:,2]=='LF']='low fat'
test.iloc[:,2][test.iloc[:,2]=='reg']='regular'
test.iloc[:,2]=test.iloc[:,2].str.lower()
outlet=test.iloc[:,6].values.reshape((1,test.shape[0]))
var_mod=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','Outlet_Size','Outlet_Location_Type','Outlet_Type']
le = LabelEncoder()
for i in var_mod:
    test[i] = test[i].factorize()[0]
    test[i] = le.fit_transform(test[i])
item=test.iloc[:,0].values.reshape((1,test.shape[0]))
test=test.iloc[:,[1,2,3,4,5,6,7,8,9,10]]
test

C:\Users\Parth Patekar\Anaconda2\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Parth Patekar\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,20.750000,0,0.007565,0,107.8622,0,0,1,0,0
1,8.300000,1,0.038428,1,87.3198,1,1,0,1,0
2,14.600000,0,0.099575,2,241.7538,2,2,0,2,1
3,7.315000,0,0.015388,0,155.0340,1,1,0,1,0
4,12.695633,1,0.118599,1,234.2300,3,3,1,2,2
5,9.800000,1,0.063817,3,117.1492,4,4,2,0,0
6,19.350000,1,0.082602,4,50.1034,5,5,1,2,3
7,12.695633,0,0.015782,4,81.0592,3,3,1,2,2
8,6.305000,1,0.123365,0,95.7436,6,6,0,1,0
9,5.985000,0,0.005698,4,186.8924,1,1,0,1,0


In [48]:
sales=lr.predict(test)
sales

array([ 1453.51658205,  1081.17641743,  3559.05261691, ...,  1728.52629153,
        3176.41636803,  1135.62694229])

In [49]:
r1=np.vstack((item,outlet))
r2=np.vstack((r1,sales))
result=pd.DataFrame(r2.T, columns=['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales'])
result.to_csv('sampleSubmission.csv',index=False)